In [1]:
import torch

### repeating samples along axis

*repeat_interleave* is equal to *cat and view*

In [3]:
a =torch.tensor([[[1,2]],[[2,3]]])
print(a,a.shape)

a.repeat_interleave(repeats=2,dim=0),torch.cat([a.unsqueeze(dim=1)]*2,dim=1).view(-1,1,2)

tensor([[[1, 2]],

        [[2, 3]]]) torch.Size([2, 1, 2])


(tensor([[[1, 2]],
 
         [[1, 2]],
 
         [[2, 3]],
 
         [[2, 3]]]),
 tensor([[[1, 2]],
 
         [[1, 2]],
 
         [[2, 3]],
 
         [[2, 3]]]))

### understanding *permute*

let's suppose there is a tensor of $[dim0, dim1, dim2, dim3]$, then we permute it to $[dim3, dim1, dim2, dim0]$,
    - **consequence**: origin value at $[a, b, c, d]$ will be switched to $[d, b, c, a]$

In [ ]:
a = torch.zeros([5,4,3,2]),
# say a=2, b=3, c=1, d=1,
a[2,3,1,1] = 1,
print("origin 1 at [2,3,1,1]: {}".format(a[2,3,1,1])),
b = a.permute(3,1,2,0),
print("permuted 1 at [3,1,2,0]: {}".format(b[1,3,1,2]))

### torch.autograd
Only when the operation in the forward phrase is **not differentiable** while you want the gradient to be pass through that you should rewrite torch.autograd, where you can define your own backward algorithm to give an approximate gradient of the **indifferentiable** operation.

### embedding

sometimes we have to create an embedding layer (*loop up layer*). The derivation from emebdding layer is straight forward: the last |n-1| dimension in embedding layer will be appended to the index tensor.

In [13]:
embedding_layer = torch.rand((10,3))
index_tensor = torch.tensor([[3,5],[0,9]]) # only tensor of dtype=torch.long works
print("1 dimensional embedding:{} of size {}\n".format(embedding_layer[index_tensor], embedding_layer[index_tensor].shape))

embedding_layer = torch.rand((10,10,3))
print("2 dimensional embedding:{} of size {}".format(embedding_layer[index_tensor], embedding_layer[index_tensor].shape))

1 dimensional embedding:tensor([[[0.9326, 0.0780, 0.4619],
         [0.7300, 0.4935, 0.7460]],

        [[0.2537, 0.5904, 0.6290],
         [0.3936, 0.9263, 0.7734]]]) of size torch.Size([2, 2, 3])

2 dimensional embedding:tensor([[[[0.3148, 0.9861, 0.3106],
          [0.2436, 0.2787, 0.1736],
          [0.2394, 0.4007, 0.8292],
          [0.7823, 0.4570, 0.8251],
          [0.0033, 0.7966, 0.3431],
          [0.1534, 0.0922, 0.7102],
          [0.5269, 0.0903, 0.5201],
          [0.4941, 0.4445, 0.9990],
          [0.9150, 0.0548, 0.5993],
          [0.2121, 0.2611, 0.8124]],

         [[0.9414, 0.2698, 0.3039],
          [0.4300, 0.0050, 0.9384],
          [0.3817, 0.5735, 0.0380],
          [0.2574, 0.4992, 0.0408],
          [0.7197, 0.6007, 0.9922],
          [0.3779, 0.3335, 0.8303],
          [0.7819, 0.6518, 0.3244],
          [0.6018, 0.1290, 0.4581],
          [0.0500, 0.8068, 0.1329],
          [0.6273, 0.7432, 0.4093]]],


        [[[0.3525, 0.9594, 0.4348],
          [0.18

### finding non-zero indices

In [9]:
a = torch.zeros(2,2)
a[1,0] = 1
a.nonzero()

tensor([[1, 0]])